In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
sliderDL = pd.read_csv('DraftLeagueSlider.csv')
sliderMLB = pd.read_csv('MLBSlider.csv')

In [3]:
sliderDL['league'] = 'Draft League'
sliderMLB['league'] = 'MLB'

In [4]:
sliderDL = sliderDL.astype(str)
sliderMLB = sliderMLB.astype(str)

In [5]:
pitchersDL = list(set(sliderDL['pitcher']))

In [6]:
slider = pd.concat([sliderDL, sliderMLB])

In [7]:
def combined_features(row):
    return row['avgvelo']+" "+row['avgspinrate']+" "+row['avghorzbreak']+" "+row['avgvertbreak']+" "+row['avgeffectvelo']

In [8]:
#fig, ax = plt.subplots(figsize=(100,100))
#sns.heatmap(cosine_sim,cmap='magma',ax=ax)

In [9]:
def get_index_from_pitcher(pitcher):
    return slider[slider.pitcher == pitcher].index[0]

In [10]:
def get_pitcher_from_index(index):
    return slider[slider.index == index].values[0][0]

In [11]:
for x in pitchersDL:
    
    try:

        slider = pd.concat([sliderDL, sliderMLB])

        slider["combined_features"] = slider.apply(combined_features, axis =1)

        cv = CountVectorizer()
        count_matrix = cv.fit_transform(slider["combined_features"])

        cosine_sim = cosine_similarity(count_matrix)

        pitcher_like = x
        slider = slider[(slider['pitcher'] == pitcher_like) | (slider['league'] == 'MLB')]

        player_index = get_index_from_pitcher(pitcher_like)

        similar_pitchers = list(enumerate(cosine_sim[player_index]))

        sorted_similar_pitchers = sorted(similar_pitchers, key=lambda x:x[1], reverse=True)

        i=0

        for pitcher in sorted_similar_pitchers:
            print(get_pitcher_from_index(pitcher[0]),", Cosine Sim: ", sorted_similar_pitchers[i][1])
            i=i+1
            if i>1:
                break
                
    except IndexError:
        print(x, ', No Similarity')
        pass

Camfield, Geoffrey , Cosine Sim:  1.0000000000000002
Kolarek, Adam , Cosine Sim:  0.31622776601683794
Wilson, Jeffrey , Cosine Sim:  1.0000000000000002
Wittgren, Nick , Cosine Sim:  0.2519763153394848
Wendell, Wyatt , Cosine Sim:  0.9999999999999999
Feliz, Neftalí , Cosine Sim:  0.31622776601683794
Kane, Thomas , Cosine Sim:  1.0000000000000002
Martin, Chris , Cosine Sim:  0.2519763153394848
Czerniawski, Zachary , Cosine Sim:  1.0000000000000002
Czerniawski, Zachary , No Similarity
Ramos, Jean Paul , Cosine Sim:  1.0000000000000002
Colina, Edwar , Cosine Sim:  0.2222222222222222
Morris, Gunner , Cosine Sim:  1.0000000000000002
Hand, Brad , Cosine Sim:  0.31622776601683794
Brewer, Michael , Cosine Sim:  1.0000000000000002
Gant, John , Cosine Sim:  0.3333333333333333
Fruit, Zach , Cosine Sim:  1.0
Frankoff, Seth , Cosine Sim:  0.42640143271122083
Kiritsis, Daniel , Cosine Sim:  1.0
Ginkel, Kevin , Cosine Sim:  0.42640143271122083
Whaley, Alec , Cosine Sim:  1.0000000000000002
Shaw, Bryan

Morrissette, Wagner , Cosine Sim:  1.0000000000000002
Jurado, Ariel , Cosine Sim:  0.2721655269759087
Pickell, Cannon , Cosine Sim:  1.0000000000000002
Crouse, Hans , Cosine Sim:  0.35355339059327373
Busse, Terry , Cosine Sim:  0.9999999999999999
Santana, Dennis , Cosine Sim:  0.30000000000000004
Smith, Chase , Cosine Sim:  1.0000000000000002
Ottavino, Adam , Cosine Sim:  0.3333333333333333
Blatter, Brock , Cosine Sim:  1.0
Blatter, Brock , No Similarity
Braithwaite, Trey , Cosine Sim:  0.9999999999999999
Brault, Steven , Cosine Sim:  0.2860387767736777
Schomberg, Will , Cosine Sim:  0.9999999999999999
Nance, Tommy , Cosine Sim:  0.21081851067789195
Olson, Mason , Cosine Sim:  1.0000000000000002
Nance, Tommy , Cosine Sim:  0.2222222222222222
Chiriboga, Chris , Cosine Sim:  1.0000000000000002
Cortes, Nestor , Cosine Sim:  0.35355339059327373
Ammerlaan, Ashe , Cosine Sim:  0.9999999999999999
Avila, Pedro , Cosine Sim:  0.31622776601683794
Pintaro, Jonathan , Cosine Sim:  0.99999999999999